In [1]:
import os
import sys
import utils
import random
import dataset
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.eager as tfe

# mode of execution
%matplotlib inline
tfe.enable_eager_execution()
#sess=tf.Session()

# load data
os.system("sh clean.sh")
X=dataset.load_dataset("./ShapeNet",render_count=1,num_of_examples=1)
Y=None;

[load_dataset] loading from ./ShapeNet
[construct_paths] parsing dir ./ShapeNet for ['.obj', '.mtl'] ...
... done
[write_renders_to_disk] writing renders to ./ShapeNet_Renders/02691156/ee0105d05a5007ab827afb33b6d5eec6 ... 
... done
[fetch_renders_from_disk] fetching renders from ./ShapeNet_Renders/02691156/ee0105d05a5007ab827afb33b6d5eec6 ... 
... done


In [2]:
# encoder network
cur_tensor=tf.convert_to_tensor(X[0],dtype=tf.float32)
print(cur_tensor.shape)
k_s = [3,3]
conv_filter_count = [96, 128, 256, 256, 256, 256]
for i in range(6): 
    ks=[7,7]if i is 0 else k_s  
    with tf.name_scope("encoding_block"):
        cur_tensor=tf.layers.conv2d(cur_tensor,filters=conv_filter_count[i],kernel_size= k_s,activation=None)
        cur_tensor=tf.layers.max_pooling2d(cur_tensor,2,2)
        cur_tensor=tf.nn.relu(cur_tensor)
        print(cur_tensor.shape)

# flatten tensor
cur_tensor=tf.contrib.layers.flatten(cur_tensor)
print(cur_tensor.shape)
cur_tensor=tf.contrib.layers.fully_connected(cur_tensor,1024,activation_fn=None)
print(cur_tensor.shape)

(1, 512, 512, 3)
(1, 255, 255, 96)
(1, 126, 126, 128)
(1, 62, 62, 256)
(1, 30, 30, 256)
(1, 14, 14, 256)
(1, 6, 6, 256)
(1, 9216)
(1, 1024)


In [5]:
# 3D_Convolutional_LSTM 
# create 4X4X4 grid of cells
N=2
cell_grid=np.array([[[tf.nn.rnn_cell.BasicLSTMCell(1024) for k in range(N)] for j in range(N)] for i in range(N)])


hidden_state=tf.zeros([1,1024])
for index, cell in np.ndenumerate(cell_grid):
    a,b=cell(cur_tensor,hidden_state)
       

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
# decoding network
# batch_size=cur_tensor.shape.as_list()[0]
# cur_tensor=tf.reshape(cur_tensor,[batch_size,2,2,2,-1])
# print(cur_tensor.shape)
k_s = [3,3,3]
deconv_filter_count = [128, 128, 128, 64, 32, 2]
for i in range(6): 
    with tf.name_scope("decoding_block"):
            cur_tensor=tf.layers.conv3d_transpose(cur_tensor,padding='VALID',filters=deconv_filter_count[i],kernel_size= k_s,activation=None)
            cur_tensor=utils.unpool(cur_tensor)
            cur_tensor=tf.nn.relu(cur_tensor)
            print(cur_tensor.shape)

In [ ]:
# serialize graph for tensorboard
os.system("rm -rf ./logs/*")
writer = tf.summary.FileWriter("./logs/")
writer.add_graph(sess.graph)
os.system("sh tensorboard.sh")